In [5]:
using Plots, Random

function rosenbrock(x::Vector{Float64})
    return (1 - x[1])^2 + 100 * (x[2] - x[1]^2)^2
end

n = 2                     # число размерностей
domain_min = -5.12        # нижняя граница
domain_max = 5.12         # верхняя граница
B = 11                    # число дискретизирующих точек (интервалы: B-1)
N = 30                    # число муравьёв
max_iter = 100            # число поколений

alpha = 1.0             # важность феромона
Q = 20.0                 # константа выделения феромона
rho = 0.9               # скорость испарения
tau0 = 1.0              # начальный уровень феромона

# Дискретизация и инициализация феромонов
boundaries = Vector{Vector{Float64}}(undef, n)
for i in 1:n
    boundaries[i] = collect(range(domain_min, domain_max, length=B))
end
pheromones = [fill(tau0, B - 1) for i in 1:n]

# Функция генерации решения для одного муравья
function generate_solution(pheromones, boundaries, alpha)
    sol = zeros(n)
    for i in 1:n
        tau_vec = pheromones[i]
        prob = [tau^alpha for tau in tau_vec]
        prob ./= sum(prob)
        r = rand(); cumulative = 0.0; chosen_interval = 0
        for j in 1:length(prob)
            cumulative += prob[j]
            if r <= cumulative
                chosen_interval = j
                break
            end
        end
        chosen_interval = (chosen_interval == 0 ? length(prob) : chosen_interval)
        b_low = boundaries[i][chosen_interval]
        b_high = boundaries[i][chosen_interval+1]
        sol[i] = rand() * (b_high - b_low) + b_low
    end
    return sol
end

# Подготовка для анимации
solutions_history = Vector{Matrix{Float64}}()   # 2D положения муравьёв
best_history = Vector{Vector{Float64}}()          # лучшая точка
best_value_history = Float64[]
pheromone_history = Vector{Any}()                 # история феромонов

global_best_solution = zeros(n)
global_best_value = Inf

# Основной цикл алгоритма с элитизмом
for iter in 1:max_iter
    solutions = [generate_solution(pheromones, boundaries, alpha) for k in 1:N]
    costs = [rastrigin(sol) for sol in solutions]

    # Элитизм: включаем глобально найденную лучшую точку, если она уже найдена
    if global_best_value < Inf
        solutions[1] = copy(global_best_solution)
        costs[1] = global_best_value
    end

    # Обновление глобального лучшего решения
    for k in 1:N
        if costs[k] < global_best_value
            global_best_value = costs[k]
            global_best_solution = copy(solutions[k])
        end
    end

    # Сохранение данных для анимации
    sol_matrix = zeros(N, 2)
    for k in 1:N
        sol_matrix[k, :] = solutions[k]
    end
    push!(solutions_history, sol_matrix)
    push!(best_history, copy(global_best_solution))
    push!(best_value_history, global_best_value)
    push!(pheromone_history, deepcopy(pheromones))

    # Обновление феромонов (с защитой от деления на 0)
    eps = 1e-10
    for i in 1:n
        delta_tau = zeros(B - 1)
        for k in 1:N
            sol_val = solutions[k][i]
            for j in 1:(B - 1)
                if sol_val >= boundaries[i][j] && sol_val < boundaries[i][j+1]
                    delta_tau[j] += Q / (costs[k] + eps)
                    break
                elseif j == B - 1 && sol_val == boundaries[i][end]
                    delta_tau[j] += Q / (costs[k] + eps)
                    break
                end
            end
        end
        for j in 1:(B - 1)
            pheromones[i][j] = (1 - rho)*pheromones[i][j] + delta_tau[j]
        end
    end
    # println("Поколение $iter: Лучший L = $(global_best_value)")
end

# Подготовка данных для графиков функции Растригина
xgrid = collect(range(domain_min, domain_max, length=100))
ygrid = collect(range(domain_min, domain_max, length=100))
zgrid = [rosenbrock([x, y]) for x in xgrid, y in ygrid]

# Центры ячеек для тепловой карты феромона
x_centers = [(boundaries[1][i] + boundaries[1][i+1]) / 2 for i in 1:(B-1)]
y_centers = [(boundaries[2][i] + boundaries[2][i+1]) / 2 for i in 1:(B-1)]
# Анимация: три графика в одном кадре (с квадратным соотношением сторон)
anim = @animate for i in 1:length(solutions_history)
    sol_mat = solutions_history[i]
    best_sol = best_history[i]

    # График p1: 2D-контуры и муравьи
    p1 = contour(xgrid, ygrid, zgrid', fill=false, levels=20,                       color=:viridis,
                 xlims=(domain_min, domain_max), ylims=(domain_min, domain_max),
                 xlabel="x", ylabel="y",
                 title="Поколение: $i | Лучший: $(round(best_value_history[i], digits=2))",
                 aspect_ratio=1)
    scatter!(p1, sol_mat[:,1], sol_mat[:,2], label="Муравьи")
    scatter!(p1, [best_sol[1]], [best_sol[2]], markershape=:circle, markersize=10,
             color=:red, label="Лучшая")

    # График p2: 3D-поверхность и муравьи
    p2 = surface(xgrid, ygrid, zgrid', alpha=0.6,
                 xlims=(domain_min, domain_max), ylims=(domain_min, domain_max),                       color=:viridis,
                 zlims=(minimum(zgrid), maximum(zgrid)),
                 xlabel="x", ylabel="y", zlabel="f(x,y)",
                 title="3D Поколение: $i",
                 aspect_ratio=1)
    z_sol = [rosenbrock([sol_mat[k,1], sol_mat[k,2]]) for k in 1:N]
    scatter3d!(p2, sol_mat[:,1], sol_mat[:,2], z_sol, markerstrokewidth=0,
               label="Муравьи")
    scatter3d!(p2, [best_sol[1]], [best_sol[2]], [rosenbrock(best_sol)],
               markershape=:circle, markersize=10, color=:red, label="Лучшая")


    # Объединяем графики: 1 строка, 3 столбца, общий размер 1200x400 (каждый график квадратный)
    plot(p1, p2, layout = (1, 2), legend = :bottomright, size=(1200,400))
end

gif(anim, "ant_colony_combined_3panels_square.gif", fps=10)
println("Объединённая анимация с 3 квадратными графиками сохранена в файле ant_colony_combined_3panels_square.gif")


Объединённая анимация с 3 квадратными графиками сохранена в файле ant_colony_combined_3panels_square.gif


┌ Info: Saved animation to /Users/d.okutin/ant_colony_combined_3panels_square.gif
└ @ Plots /Users/d.okutin/.julia/packages/Plots/kLeqV/src/animation.jl:156


In [6]:
function schwefel(x)
    return 418.9829*2 - (x[1]*sin(sqrt(abs(x[1]))) + x[2]*sin(sqrt(abs(x[2]))))
end

using Plots, Random

n = 2  
domain_min = -5.12    
domain_max = 5.12         
B = 11                    
N = 30                  
max_iter = 100 

alpha = 1.0
Q = 20.0
rho = 0.9
tau0 = 1.0 

# Дискретизация и инициализация феромонов
boundaries = Vector{Vector{Float64}}(undef, n)
for i in 1:n
    boundaries[i] = collect(range(domain_min, domain_max, length=B))
end
pheromones = [fill(tau0, B - 1) for i in 1:n]

# Подготовка для анимации
solutions_history = Vector{Matrix{Float64}}()   # 2D положения муравьёв
best_history = Vector{Vector{Float64}}()          # лучшая точка
best_value_history = Float64[]
pheromone_history = Vector{Any}()                 # история феромонов

global_best_solution = zeros(n)
global_best_value = Inf

# Основной цикл алгоритма с элитизмом
for iter in 1:max_iter
    solutions = [generate_solution(pheromones, boundaries, alpha) for k in 1:N]
    costs = [schwefel(sol) for sol in solutions]

    # Элитизм: включаем глобально найденную лучшую точку, если она уже найдена
    if global_best_value < Inf
        solutions[1] = copy(global_best_solution)
        costs[1] = global_best_value
    end

    # Обновление глобального лучшего решения
    for k in 1:N
        if costs[k] < global_best_value
            global_best_value = costs[k]
            global_best_solution = copy(solutions[k])
        end
    end

    # Сохранение данных для анимации
    sol_matrix = zeros(N, 2)
    for k in 1:N
        sol_matrix[k, :] = solutions[k]
    end
    push!(solutions_history, sol_matrix)
    push!(best_history, copy(global_best_solution))
    push!(best_value_history, global_best_value)
    push!(pheromone_history, deepcopy(pheromones))

    # Обновление феромонов (с защитой от деления на 0)
    eps = 1e-10
    for i in 1:n
        delta_tau = zeros(B - 1)
        for k in 1:N
            sol_val = solutions[k][i]
            for j in 1:(B - 1)
                if sol_val >= boundaries[i][j] && sol_val < boundaries[i][j+1]
                    delta_tau[j] += Q / (costs[k] + eps)
                    break
                elseif j == B - 1 && sol_val == boundaries[i][end]
                    delta_tau[j] += Q / (costs[k] + eps)
                    break
                end
            end
        end
        for j in 1:(B - 1)
            pheromones[i][j] = (1 - rho)*pheromones[i][j] + delta_tau[j]
        end
    end
    # println("Поколение $iter: Лучший L = $(global_best_value)")
end

# Подготовка данных для графиков функции Растригина
xgrid = collect(range(domain_min, domain_max, length=100))
ygrid = collect(range(domain_min, domain_max, length=100))
zgrid = [schwefel([x, y]) for x in xgrid, y in ygrid]

# Центры ячеек для тепловой карты феромона
x_centers = [(boundaries[1][i] + boundaries[1][i+1]) / 2 for i in 1:(B-1)]
y_centers = [(boundaries[2][i] + boundaries[2][i+1]) / 2 for i in 1:(B-1)]
# Анимация: три графика в одном кадре (с квадратным соотношением сторон)
anim = @animate for i in 1:length(solutions_history)
    sol_mat = solutions_history[i]
    best_sol = best_history[i]

    # График p1: 2D-контуры и муравьи
    p1 = contour(xgrid, ygrid, zgrid', fill=false, levels=20,                       color=:viridis,
                 xlims=(domain_min, domain_max), ylims=(domain_min, domain_max),
                 xlabel="x", ylabel="y",
                 title="Поколение: $i | Лучший : $(round(best_value_history[i], digits=2))",
                 aspect_ratio=1)
    scatter!(p1, sol_mat[:,1], sol_mat[:,2], label="Муравьи")
    scatter!(p1, [best_sol[1]], [best_sol[2]], markershape=:star5, markersize=10,
             color=:red, label="Лучшая")

    # График p2: 3D-поверхность и муравьи
    p2 = surface(xgrid, ygrid, zgrid', alpha=0.6,
                 xlims=(domain_min, domain_max), ylims=(domain_min, domain_max),                       color=:viridis,
                 zlims=(minimum(zgrid), maximum(zgrid)),
                 xlabel="x", ylabel="y", zlabel="f(x,y)",
                 title="3D Поколение: $i",
                 aspect_ratio=1)
    z_sol = [schwefel([sol_mat[k,1], sol_mat[k,2]]) for k in 1:N]
    scatter3d!(p2, sol_mat[:,1], sol_mat[:,2], z_sol, markerstrokewidth=0,
               label="Муравьи")
    scatter3d!(p2, [best_sol[1]], [best_sol[2]], [schwefel(best_sol)],
               markershape=:circle, markersize=10, color=:red, label="Лучшая")


    # Объединяем графики: 1 строка, 3 столбца, общий размер 1200x400 (каждый график квадратный)
    plot(p1, p2, layout = (1, 2), legend = :bottomright, size=(1200,400))
end

gif(anim, "ant_colony_combined_3panels_square_shwefel.gif", fps=10)
println("Объединённая анимация с 3 квадратными графиками сохранена в файле ant_colony_combined_3panels_square_shwefel.gif")


Объединённая анимация с 3 квадратными графиками сохранена в файле ant_colony_combined_3panels_square_shwefel.gif


┌ Info: Saved animation to /Users/d.okutin/ant_colony_combined_3panels_square_shwefel.gif
└ @ Plots /Users/d.okutin/.julia/packages/Plots/kLeqV/src/animation.jl:156


In [8]:
function rastrigin(x)
    return 20 + x[1]^2 + x[2]^2 - 10 * (cos(2π * x[1]) + cos(2π * x[2]))
end

using Plots, Random

n = 2  
domain_min = -5.12    
domain_max = 5.12         
B = 11                    
N = 30                  
max_iter = 100 

alpha = 1.0
Q = 20.0
rho = 0.9
tau0 = 1.0 

# Дискретизация и инициализация феромонов
boundaries = Vector{Vector{Float64}}(undef, n)
for i in 1:n
    boundaries[i] = collect(range(domain_min, domain_max, length=B))
end
pheromones = [fill(tau0, B - 1) for i in 1:n]

# Подготовка для анимации
solutions_history = Vector{Matrix{Float64}}()   # 2D положения муравьёв
best_history = Vector{Vector{Float64}}()          # лучшая точка
best_value_history = Float64[]
pheromone_history = Vector{Any}()                 # история феромонов

global_best_solution = zeros(n)
global_best_value = Inf

# Основной цикл алгоритма с элитизмом
for iter in 1:max_iter
    solutions = [generate_solution(pheromones, boundaries, alpha) for k in 1:N]
    costs = [rastrigin(sol) for sol in solutions]

    # Элитизм: включаем глобально найденную лучшую точку, если она уже найдена
    if global_best_value < Inf
        solutions[1] = copy(global_best_solution)
        costs[1] = global_best_value
    end

    # Обновление глобального лучшего решения
    for k in 1:N
        if costs[k] < global_best_value
            global_best_value = costs[k]
            global_best_solution = copy(solutions[k])
        end
    end

    # Сохранение данных для анимации
    sol_matrix = zeros(N, 2)
    for k in 1:N
        sol_matrix[k, :] = solutions[k]
    end
    push!(solutions_history, sol_matrix)
    push!(best_history, copy(global_best_solution))
    push!(best_value_history, global_best_value)
    push!(pheromone_history, deepcopy(pheromones))

    # Обновление феромонов (с защитой от деления на 0)
    eps = 1e-10
    for i in 1:n
        delta_tau = zeros(B - 1)
        for k in 1:N
            sol_val = solutions[k][i]
            for j in 1:(B - 1)
                if sol_val >= boundaries[i][j] && sol_val < boundaries[i][j+1]
                    delta_tau[j] += Q / (costs[k] + eps)
                    break
                elseif j == B - 1 && sol_val == boundaries[i][end]
                    delta_tau[j] += Q / (costs[k] + eps)
                    break
                end
            end
        end
        for j in 1:(B - 1)
            pheromones[i][j] = (1 - rho)*pheromones[i][j] + delta_tau[j]
        end
    end
    # println("Поколение $iter: Лучший L = $(global_best_value)")
end

# Подготовка данных для графиков функции Растригина
xgrid = collect(range(domain_min, domain_max, length=100))
ygrid = collect(range(domain_min, domain_max, length=100))
zgrid = [rastrigin([x, y]) for x in xgrid, y in ygrid]

# Центры ячеек для тепловой карты феромона
x_centers = [(boundaries[1][i] + boundaries[1][i+1]) / 2 for i in 1:(B-1)]
y_centers = [(boundaries[2][i] + boundaries[2][i+1]) / 2 for i in 1:(B-1)]
# Анимация: три графика в одном кадре (с квадратным соотношением сторон)
anim = @animate for i in 1:length(solutions_history)
    sol_mat = solutions_history[i]
    best_sol = best_history[i]

    # График p1: 2D-контуры и муравьи
    p1 = contour(xgrid, ygrid, zgrid', fill=false, levels=20,                       color=:viridis,
                 xlims=(domain_min, domain_max), ylims=(domain_min, domain_max),
                 xlabel="x", ylabel="y",
                 title="Поколение: $i | Лучший : $(round(best_value_history[i], digits=2))",
                 aspect_ratio=1)
    scatter!(p1, sol_mat[:,1], sol_mat[:,2], label="Муравьи")
    scatter!(p1, [best_sol[1]], [best_sol[2]], markershape=:circle, markersize=10,
             color=:red, label="Лучшая")

    # График p2: 3D-поверхность и муравьи
    p2 = surface(xgrid, ygrid, zgrid', alpha=0.6,
                 xlims=(domain_min, domain_max), ylims=(domain_min, domain_max),                       color=:viridis,
                 zlims=(minimum(zgrid), maximum(zgrid)),
                 xlabel="x", ylabel="y", zlabel="f(x,y)",
                 title="3D Поколение: $i",
                 aspect_ratio=1)
    z_sol = [rastrigin([sol_mat[k,1], sol_mat[k,2]]) for k in 1:N]
    scatter3d!(p2, sol_mat[:,1], sol_mat[:,2], z_sol, markerstrokewidth=0,
               label="Муравьи")
    scatter3d!(p2, [best_sol[1]], [best_sol[2]], [rastrigin(best_sol)],
               markershape=:circle, markersize=10, color=:red, label="Лучшая")


    # Объединяем графики: 1 строка, 3 столбца, общий размер 1200x400 (каждый график квадратный)
    plot(p1, p2, layout = (1, 2), legend = :bottomright, size=(1200,400))
end

gif(anim, "ant_colony_combined_3panels_square_rastrigin.gif", fps=10)
println("Объединённая анимация с 3 квадратными графиками сохранена в файле ant_colony_combined_3panels_square_rastrigin.gif")


Объединённая анимация с 3 квадратными графиками сохранена в файле ant_colony_combined_3panels_square_rastrigin.gif


┌ Info: Saved animation to /Users/d.okutin/ant_colony_combined_3panels_square_rastrigin.gif
└ @ Plots /Users/d.okutin/.julia/packages/Plots/kLeqV/src/animation.jl:156
